# Developing JuliaFEM

Author(s): Jukka Aho

**Abstract**: Developer notes. In this notebook we give general guidelines how to implement things to JuliaFEM.

In short, we have

**Element**: object which holds basis functions and fields. One is able to interpolate fields using element. Introduction of new basis functions needs new elements. Typical elements: Lagrange elements, hierarchical elements, etc.

**Equation**: object which defines some field equation which needs to be solve. One element can have several equations but not vice versa. Typical equations: Poisson equation $\Delta u = f$, elasticity equation $\nabla \cdot \sigma = f$, etc.

**Problem**: object which maps equations to elements. For example, HeatProblem which maps Poisson equation to Lagrange elements or ElasticityProblem which maps elasticity equation to Lagrange elements.

**Solver**: object which takes one or more problems, solves them using some strategy (iterative methods, multigrid, direct methods, ...) and updates corresponding fields to elements.

Moreover, we have (will have) **`test_element`**, **`test_equation`**, **`test_problem`** and **`test_solver`** which can be used to test that implementation has all necessary things defined.

In [39]:
using Logging
using FactCheck
Logging.configure(level=DEBUG)

Logger(root,DEBUG,Base.PipeEndpoint(open, 0 bytes waiting),root)

## Developing own element

Element contains basis functions and one or several fieldsets.

Minimum requirements for element:
- subclass it from Element, unless want to implement everything by youself
- define basis and partial derivatives of it, because we need to interpolate over it
- give connectivity information, how this element is connected to other elements

Test the element using ``test_element`` function. It it passes, the element implementation should be fine.

In [40]:
using JuliaFEM: Element, Field, FieldSet, Basis

Here's the basic implementation for element:

In [41]:
type MyQuad4 <: Element
    connectivity :: Array{Int, 1}
    basis :: Basis
    fields :: Dict{Symbol, FieldSet}
end

Default constructor takes only connectivity information as input argument. Most important thing here is to define `Basis` which is used to interpolate fields.

In [42]:
function MyQuad4(connectivity)
    h(xi) = [
        (1-xi[1])*(1-xi[2])/4
        (1+xi[1])*(1-xi[2])/4
        (1+xi[1])*(1+xi[2])/4
        (1-xi[1])*(1+xi[2])/4]
    dh(xi) = [
        -(1-xi[2])/4.0   -(1-xi[1])/4.0
         (1-xi[2])/4.0   -(1+xi[1])/4.0
         (1+xi[2])/4.0    (1+xi[1])/4.0
        -(1+xi[2])/4.0    (1-xi[1])/4.0]
    basis = Basis(h, dh)
    MyQuad4(connectivity, basis, Dict())
end

MyQuad4

Also some basic information like number of basis functions and element dimension is needed.

In [43]:
JuliaFEM.get_number_of_basis_functions(el::Type{MyQuad4}) = 4
JuliaFEM.get_element_dimension(el::Type{MyQuad4}) = 2

get_element_dimension (generic function with 7 methods)

Next we check that everything is well defined:

In [44]:
using JuliaFEM: test_element
test_element(MyQuad4)

20-loka 15:49:17:INFO:root:Testing element MyQuad4


If `test_element` passes, elements *interface* should be well defined. After building element, one can interpolate things in it. Here we create three new fieldsets `temperature`, `geometry` and `heat coefficient`, add some values for them in time $t=0.0$ and $t=1.0$ and interpolate:

In [45]:
using JuliaFEM: FieldSet, Field, interpolate, dinterpolate
element = MyQuad4([1, 2, 3, 4])

geometry_field = Field(0.0, Vector[])  # Create empty field at time t=0.0
push!(geometry_field, [ 0.0, 0.0, 0.0])  # push some values for field
push!(geometry_field, [10.0, 0.0, 0.0])
push!(geometry_field, [10.0, 1.0, 0.0])
push!(geometry_field, [ 0.0, 1.0, 0.0])
geometry_fieldset = FieldSet("geometry")  # create fieldset "geometry"
push!(geometry_fieldset, geometry_field)  # add field to fieldset
push!(element, geometry_fieldset) # add fieldset to element

temperature_fieldset = FieldSet("temperature")
push!(temperature_fieldset, Field(0.0, [0.0, 0.0, 0.0, 0.0]))
push!(temperature_fieldset, Field(1.0, [1.0, 2.0, 3.0, 4.0]))
push!(element, temperature_fieldset)

heat_coefficient_fieldset = FieldSet("heat coefficient")
push!(heat_coefficient_fieldset, Field(0.0, 2))
push!(heat_coefficient_fieldset, Field(1.0, 3))
push!(element, heat_coefficient_fieldset)

JuliaFEM.FieldSet(symbol("heat coefficient"),JuliaFEM.Field[JuliaFEM.Field{Int64}(0.0,1,2),JuliaFEM.Field{Int64}(1.0,1,3)])

In [46]:
# temperature at the middle poinf of the element, 1/4*(1+2+3+4) at t=0.5
interpolate(element, "temperature", [0.0, 0.0], 0.5)

1.25

In [47]:
# geometry midpoint of element
interpolate(element, "geometry", [0.0, 0.0], 0.0)

3-element Array{Float64,1}:
 5.0
 0.5
 0.0

In [48]:
# interpolate derivatives works too
dinterpolate(element, "geometry", [0.0, 0.0], 0.0)

3x2 Array{Float64,2}:
 5.0  0.0
 0.0  0.5
 0.0  0.0

In [49]:
# interpolating scalar -> scalar.
interpolate(element, "heat coefficient", [0.0, 0.0], 0.5)

2.5

In [50]:
interpolate(element, "heat coefficient", [0.0, 0.0], Inf)

3

### Summary of developing own elements

Element itself if not calculating anything but only stores fields and basis functions so that the fields can be interpolated. We will provide command `test_element` which will ensure that everything necessary is defined. While lot of things needs to be defined, by subclassing from `Element` most of these are already defined, thanks to multiple dispatch.

## Developing own equation

Let's consider a Laplace equation
\begin{align}
\Delta{u} &= 0 && \text{on } \Omega \\
\frac{\partial u}{\partial n} &= g && \text{on } \Gamma_{\mathrm{N}}
\end{align}

Weak form is, find $u\in\mathcal{U}$ such that
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x = \int_{\Gamma_{\mathrm{N}}}g v\,\mathrm{d}s \quad \forall v\in\mathcal{V}.
\end{equation}

Minimum requirements for equation: 
- subclass from Equation, if not want to implement from scratch
- it needs to have lhs and rhs functions
- provide the name of the unknown field variable trying to solve
- default constructor takes the element as input argument

Now we have function `test_equation`, which we can use to test that everything is working as expected. 

Again thanks to multiple dispatch, you are free to code your weak form however you want as long as it returns lhs and rhs sides for element dofs. This kind of freedom gives good opportunities to wrap e.g. Fortran code from some other projects. And again we have some suggestions ad following these ideas you get a lot of stuff for free. First we look the left hand side of the equation, that is,
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x
\end{equation}

In [51]:
using JuliaFEM: Equation, IntegrationPoint, Quad4, get_unknown_field_name

abstract Heat <: Equation

# it's important to define for which fieldset to save unknown values when solving
JuliaFEM.get_unknown_field_name(eq::Heat) = symbol("temperature")

get_unknown_field_name (generic function with 4 methods)

Our basic data type often looks something like this:

In [52]:
"""
Diffusive heat transfer for 4-node bilinear element.
"""
type DC2D4 <: Heat
    element :: Quad4
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

We must provide default constructor which takes element as input argument:

In [53]:
function DC2D4(element::Quad4)
    integration_points = [
        IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0)]
    push!(element, FieldSet("temperature"))
    DC2D4(element, integration_points, [])
end

DC2D4

Now the actual implementation for $\int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x$:

In [54]:
using JuliaFEM: get_element, get_dbasisdX, has_lhs, has_rhs

"""
Left hand side defined in integration point
"""
function JuliaFEM.get_lhs(eq::DC2D4, ip, t)
    el = get_element(eq)
    dNdX = get_dbasisdX(el, ip.xi, t)
    hc = interpolate(el, "temperature thermal conductivity", ip.xi, t)
    return dNdX*hc*dNdX'
end
JuliaFEM.has_lhs(eq::DC2D4) = true

has_lhs (generic function with 4 methods)

And that's it. If we want to play with this formulation, we must create element and assign this equation for it:

In [55]:
using JuliaFEM: integrate, integrate_lhs, integrate_rhs
element = Quad4([1, 2, 3, 4])
fieldset1 = FieldSet("geometry")
field1 = Field(0.0, Vector[[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]])
push!(fieldset1, field1)
fieldset2 = FieldSet("temperature thermal conductivity")
push!(fieldset2, Field(0.0, 6.0))
push!(element, fieldset1)
push!(element, fieldset2)
equation = DC2D4(element)
integrate_lhs(equation, 1.0)

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [56]:
JuliaFEM.has_lhs(equation)

true

If rhs or lhs is not defined, integration returns nothing.

In [57]:
integrate_rhs(equation, 1.0) == nothing

true

Next heat flux on boundary:

In [58]:
using JuliaFEM: get_basis, Seg2

"""
Diffusive heat transfer for 2-node linear segment.
"""
type DC2D2 <: Heat
    element :: Seg2
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

function DC2D2(element::Seg2)
    integration_points = [
        IntegrationPoint([0.0], 2.0)]
    push!(element, FieldSet("temperature"))
    DC2D2(element, integration_points, [])
end

"""
Right hand side defined in integration point
"""
function JuliaFEM.get_rhs(eq::DC2D2, ip, t)
    el = get_element(eq)
    h = get_basis(el, ip.xi)
    #f = el["temperature flux"]
    f = interpolate(el, "temperature flux", ip.xi, t)
    return h*f
end
JuliaFEM.has_rhs(eq::DC2D2) = true

has_rhs (generic function with 4 methods)

In [59]:
element = Seg2([1, 2])
push!(element, FieldSet("geometry", [Field(0.0, Vector[[0.0,0.0], [0.0,1.0]])]))
push!(element, FieldSet("temperature flux", [Field(0.0, 100.0)]))
equation = DC2D2(element)
integrate_rhs(equation, 1.0)

2-element Array{Float64,1}:
 50.0
 50.0

## Defining own problem

- main object: takes a set of elements and maps corresponding field equations to them

In [60]:
using JuliaFEM: Problem, get_equation, get_dimension

type PlaneHeatProblem <: Problem
    equations :: Array{Equation, 1}
end
PlaneHeatProblem() = PlaneHeatProblem([])

PlaneHeatProblem

In [61]:
JuliaFEM.get_dimension(pr::Type{PlaneHeatProblem}) = 1
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Quad4}) = DC2D4
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Seg2}) = DC2D2

get_equation (generic function with 6 methods)

Our solution procedure so far is therefore

In [62]:
using JuliaFEM: get_connectivity, set_global_dofs!, get_global_dofs
using JuliaFEM: add_element!, get_equations, get_matrix_dimension, calculate_global_dofs
using JuliaFEM: assign_global_dofs!, get_lhs, get_rhs

# create elements and add necessary properties like connectivity and geometry
el1 = Quad4([2, 3, 4, 5])
fs1geom = FieldSet("geometry")
push!(fs1geom, Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]]))
fs1temp = FieldSet("temperature thermal conductivity")
push!(fs1temp, Field(0.0, 6.0))
push!(el1, fs1geom)
push!(el1, fs1temp)

el2 = Seg2([2, 3])
fs2geom = FieldSet("geometry")
push!(fs2geom, Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]]))
fs2temp = FieldSet("temperature flux")
push!(fs2temp, Field(1.0, 600.0))
push!(el2, fs2geom)
push!(el2, fs2temp)

problem = PlaneHeatProblem()
push!(problem, el1)
push!(problem, el2)

dofmap = calculate_global_dofs(problem)
assign_global_dofs!(problem, dofmap)

t = 1.0
A = sparse(get_lhs(problem, t)...)
b = sparsevec(get_rhs(problem, t)..., size(A, 1))

20-loka 15:49:17:DEBUG:root:total dofs: 4


4x1 sparse matrix with 2 Float64 entries:
	[1, 1]  =  300.0
	[2, 1]  =  300.0

In [63]:
fdofs = [1, 2]
A[fdofs, fdofs] \ b[fdofs]

2x1 sparse matrix with 2 Float64 entries:
	[1, 1]  =  100.0
	[2, 1]  =  100.0

We still need to consider Dirichlet boundary conditions:
\begin{align}
u &= u_0 && \text{on } \Gamma_{\mathrm{D}} \\
\end{align}

In [64]:
using JuliaFEM: DirichletProblem

# create elements and add necessary properties like connectivity and geometry
el3 = Seg2([4, 5])
push!(el3, FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]])]))

bc1 = DirichletProblem()
push!(bc1, el3)

1-element Array{JuliaFEM.DirichletEquation,1}:
 JuliaFEM.DBC2D2(JuliaFEM.Seg2([4,5],JuliaFEM.Basis(basis,j),Dict(symbol("reaction force")=>JuliaFEM.FieldSet(symbol("reaction force"),JuliaFEM.Field[]),:geometry=>JuliaFEM.FieldSet(:geometry,JuliaFEM.Field[JuliaFEM.Field{Array{Array{T,1},1}}(0.0,1,Array{T,1}[[0.0,0.0],[0.0,1.0]])]))),[JuliaFEM.IntegrationPoint([-0.5773502691896257],1.0,Dict{Any,Any}()),JuliaFEM.IntegrationPoint([0.5773502691896257],1.0,Dict{Any,Any}())],Int64[],fieldval)

In [65]:
assign_global_dofs!(bc1, dofmap)

# integrate and assembly
t = 1.0
A2 = sparse(get_lhs(bc1, t)...)
b2 = sparsevec(get_rhs(bc1, t)...)

4x1 sparse matrix with 2 Float64 entries:
	[3, 1]  =  0.0
	[4, 1]  =  0.0

Now we have two problems defined, 

In [66]:
full(A)

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [67]:
full(b)

4x1 Array{Float64,2}:
 300.0
 300.0
   0.0
   0.0

In [68]:
full(A2)

4x4 Array{Float64,2}:
 0.0  0.0  0.0       0.0     
 0.0  0.0  0.0       0.0     
 0.0  0.0  0.333333  0.166667
 0.0  0.0  0.166667  0.333333

In [69]:
full(b2)

4x1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0

In [70]:
Atot = [A A2; A2 zeros(A2)]

8x8 sparse matrix with 24 Float64 entries:
	[1, 1]  =  4.0
	[2, 1]  =  -1.0
	[3, 1]  =  -2.0
	[4, 1]  =  -1.0
	[1, 2]  =  -1.0
	[2, 2]  =  4.0
	[3, 2]  =  -1.0
	[4, 2]  =  -2.0
	[1, 3]  =  -2.0
	[2, 3]  =  -1.0
	⋮
	[8, 3]  =  0.166667
	[1, 4]  =  -1.0
	[2, 4]  =  -2.0
	[3, 4]  =  -1.0
	[4, 4]  =  4.0
	[7, 4]  =  0.166667
	[8, 4]  =  0.333333
	[3, 7]  =  0.333333
	[4, 7]  =  0.166667
	[3, 8]  =  0.166667
	[4, 8]  =  0.333333

In [71]:
btot = [b; b2]

8x1 sparse matrix with 4 Float64 entries:
	[1, 1]  =  300.0
	[2, 1]  =  300.0
	[7, 1]  =  0.0
	[8, 1]  =  0.0

Problem is that now are total matrix Atot has zero rows which needs to be removed.

In [72]:
full(Atot)

8x8 Array{Float64,2}:
  4.0  -1.0  -2.0       -1.0       0.0  0.0  0.0       0.0     
 -1.0   4.0  -1.0       -2.0       0.0  0.0  0.0       0.0     
 -2.0  -1.0   4.0       -1.0       0.0  0.0  0.333333  0.166667
 -1.0  -2.0  -1.0        4.0       0.0  0.0  0.166667  0.333333
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.333333   0.166667  0.0  0.0  0.0       0.0     
  0.0   0.0   0.166667   0.333333  0.0  0.0  0.0       0.0     

In [73]:
r = unique(rowvals(Atot))
println("Non-zero rows: $r")
xtot = zeros(btot)
F = lufact(Atot[r,r])
s = full(btot[r])
xtot[r] = F \ s
full(xtot)

Non-zero rows: [1,2,3,4,7,8]

8x1 Array{Float64,2}:
 100.0
 100.0
   0.0
   0.0
   0.0
   0.0
 600.0
 600.0

## Developing own solver

Last part. Defining own solver.

- takes a set of problems (typically main problem + boundary problems)
- solves them, updates fields

In [74]:
using JuliaFEM: Solver, get_problems

""" Simple solver for educational purposes. """
type SimpleSolver <: Solver
    problems
end

""" Default initializer. """
function SimpleSolver()
    SimpleSolver(Problem[])
end

"""
Call solver to solve a set of problems.

This is simple serial solver for demonstration purposes. It handles the most
common situation, i.e., some main field problem and it's Dirichlet boundary.
"""
function call(solver::SimpleSolver, t)
    problems = get_problems(solver)
    problem1 = problems[1]
    problem2 = problems[2]

    # calculate order of degrees of freedom in global matrix
    # and set the ordering to problems
    dofmap = calculate_global_dofs(problem1)
    assign_global_dofs!(problem1, dofmap)
    assign_global_dofs!(problem2, dofmap)

    # assemble problem 1
    A1 = sparse(get_lhs(problem1, t)...)
    b1 = sparsevec(get_rhs(problem1, t)..., size(A1, 1))

    # assemble problem 2
    A2 = sparse(get_lhs(problem2, t)...)
    b2 = sparsevec(get_rhs(problem2, t)..., size(A2, 1))
    
    # make one monolithic assembly
    A = [A1 A2; A2' zeros(A2)]
    b = [b1; b2]

    # solve problem
    nz = unique(rowvals(A))
    x = zeros(b)
    x[nz] = lufact(A[nz,nz]) \ full(b[nz])

    # get "problem-wise" solution vectors
    x1 = x[1:length(b1)]
    x2 = x[length(b1)+1:end]

    # check residual
    R1 = A1*x1 - b1
    R2 = A2*x2 - b2
    println("Residual norm: $(norm(R1+R2))")

    # update field for elements in problem 1
    for equation in get_equations(problem1)
        gdofs = get_global_dofs(equation)
        element = get_element(equation)
        field_name = get_unknown_field_name(equation)  # field we are solving
        field = Field(t, full(x1[gdofs])[:])
        push!(element[field_name], field)
    end

    # update field for elements in problem 2
    for equation in get_equations(problem2)
        gdofs = get_global_dofs(equation)
        element = get_element(equation)
        field_name = get_unknown_field_name(equation)
        field = Field(t, full(x2[gdofs]))
        push!(element[field_name], field)
    end
end

call (generic function with 1270 methods)

## Summary

Next we collect all things together and solve Poisson equation in domain $\Omega = [0,1]\times[0,1]$ with some Neumann boundary on $\Gamma_1$ and Dirichlet boundary on $\Gamma_2$.

In [75]:
# Define Problem 1:
# - Field function: Laplace equation Δu=0 in Ω={u∈R²|(x,y)∈[0,1]×[0,1]}
# - Neumann boundary on Γ₁={0<=x<=1, y=0}, ∂u/∂n=600 on Γ₁

el1 = Quad4([1, 2, 3, 4])
push!(el1, FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]])]))
push!(el1, FieldSet("temperature thermal conductivity", [Field(0.0, 6.0)]))

el2 = Seg2([1, 2])
push!(el2, FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0]])]))

# Boundary load, linear ramp 0 -> 600 at time 0 -> 1
load = FieldSet("temperature flux")
push!(load, Field(0.0, 0.0))
push!(load, Field(1.0, 600.0))
push!(el2, load)

problem1 = PlaneHeatProblem()
push!(problem1, el1)
push!(problem1, el2)

# Define Problem 2:
# - Dirichlet boundary Γ₂={0<=x<=1, y=1}, u=0 on Γ₂

el3 = Seg2([3, 4])
push!(el3, FieldSet("geometry", [Field(0.0, Vector[[1.0, 1.0], [0.0, 1.0]])]))

problem2 = DirichletProblem()
push!(problem2, el3)

# Create a solver for a set of problems
solver = SimpleSolver()
push!(solver, problem1)
push!(solver, problem2)

# Solve problem at time t=1.0 and update fields
call(solver, 1.0)

# Postprocess.
# Interpolate temperature field along boundary of Γ₁ at time t=1.0
xi = linspace([-1.0], [1.0], 5)
X = interpolate(el2, "geometry", xi, 1.0)
T = interpolate(el2, "temperature", xi, 1.0)
println(X)
println(T)

20-loka 15:49:17:DEBUG:root:total dofs: 4


In [76]:
@fact mean(T) --> roughly(100.0)

Success :: (line:-1) :: fact was true
  Expression: mean(T) --> roughly(100.0)
    Expected: 100.0
    Occurred: 100.00000000000003


Residual norm: 9.845568954283847e-14
Array{T,1}[[0.0,0.0],[0.25,0.0],[0.5,0.0],[0.75,0.0],[1.0,0.0]]
[100.00000000000003,100.00000000000003,100.00000000000003,100.00000000000003,100.00000000000003]


## TODO

- dynamics
- different basis for trial and test (Petrov-Galerkin)

## Frequently asked questions

Any question about data structures, better ideas and improvements are very welcome.